# Machine Learning in Python - Deep Learning with CNNs

Using a convolutional network fro the MNIST problem - based on [https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py)

In [2]:
import numpy as np
np.random.seed(1337)  # for reproducibility
import pandas as pd

from IPython.display import display, HTML, Image
from IPython.display import SVG

from sklearn import preprocessing 
from sklearn import metrics
from sklearn.model_selection import train_test_split

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.utils.vis_utils import model_to_dot

In [3]:
%qtconsole

## Dataset Pre-Processing

Read in the mnist dataset and prepare it for learning

In [4]:
abt = pd.read_csv("../MNIST_Data/mnist_train_small.csv", encoding = "ISO-8859-1")
# Put all but the target variable into the descriptive features array
X = abt[abt.columns.difference(["value"])]
Y = abt["value"]

# Use a range scaling to scale all variables to between 0 and 1
min_max_scaler = preprocessing.MinMaxScaler()
cols = X.columns
X = pd.DataFrame(min_max_scaler.fit_transform(X), columns = cols) # Watch out for putting back in columns here

# Perfrom split to train, validation, test
x_train_plus_valid, x_test, y_train_plus_valid, y_test = train_test_split(X, Y, random_state=0, test_size = 0.30, train_size = 0.7)
x_train, x_valid, y_train, y_valid = train_test_split(x_train_plus_valid, y_train_plus_valid, random_state=0, test_size = 0.199/0.7, train_size = 0.5/0.7)

# convert to numpy arrays
x_train = x_train.as_matrix()
x_valid = x_valid.as_matrix()
x_test = x_test.as_matrix()

y_train_cat = y_train.copy()
y_valid_cat = y_valid.copy()
y_test_cat = y_test.copy()

y_train = y_train.as_matrix()
y_valid = y_valid.as_matrix()
y_test = y_test.as_matrix()

C:\Users\aksunbul\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  app.launch_new_instance()
C:\Users\aksunbul\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\aksunbul\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\aksunbul\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\aksunbul\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\aksunbul\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: FutureWarning: Method .as_matrix will be removed in a future version. Use .values ins

Prepare the data for input to a convolutional network

In [5]:
num_classes = 10

# input image dimensions
img_rows, img_cols = 28, 28

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_valid = x_valid.reshape(x_valid.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_valid = x_valid.astype('float32')
x_test = x_test.astype('float32')

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_valid.shape[0], 'valid samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_valid = keras.utils.to_categorical(y_valid, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (5000, 28, 28, 1)
5000 train samples
1991 valid samples
3000 test samples


## Model Building

Set up the model

In [7]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 input_shape=input_shape))
model.add(Activation("relu"))
model.add(Conv2D(64, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
# DROPOUT EXPLANATION
# https://medium.com/@amarbudhiraja/https-medium-com-amarbudhiraja-learning-less-to-learn-better-dropout-in-deep-machine-learning-74334da4bfc5
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

Train the model

In [8]:
batch_size = 128
epochs = 50

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.adam(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_valid, y_valid))

Train on 5000 samples, validate on 1991 samples
Epoch 1/50
5000/5000 [==============================] - 19s 4ms/step - loss: 1.1588 - acc: 0.6170 - val_loss: 0.5378 - val_acc: 0.8453
Epoch 2/50
5000/5000 [==============================] - 17s 3ms/step - loss: 0.6066 - acc: 0.8100 - val_loss: 0.3812 - val_acc: 0.8850
Epoch 3/50
5000/5000 [==============================] - 17s 3ms/step - loss: 0.4638 - acc: 0.8548 - val_loss: 0.3311 - val_acc: 0.8980
Epoch 4/50
5000/5000 [==============================] - 18s 4ms/step - loss: 0.3862 - acc: 0.8780 - val_loss: 0.2784 - val_acc: 0.9126
Epoch 5/50
5000/5000 [==============================] - 18s 4ms/step - loss: 0.3164 - acc: 0.9020 - val_loss: 0.2519 - val_acc: 0.9196
Epoch 6/50
5000/5000 [==============================] - 20s 4ms/step - loss: 0.2773 - acc: 0.9142 - val_loss: 0.2335 - val_acc: 0.9272
Epoch 7/50
5000/5000 [==============================] - 19s 4ms/step - loss: 0.2607 - acc: 0.9140 - val_loss: 0.2218 - val_acc: 0.9292
Epoch 8

In [9]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.2400116385788036
Test accuracy: 0.954


Evaluate the model

In [10]:
from sklearn import metrics
print("****** Test Data ********")

# Make a set of predictions for the validation data
y_pred = model.predict_classes(np.asfarray(x_test))

# Print performance details
print(metrics.classification_report(y_test_cat, y_pred))

# Print confusion matrix
print("Confusion Matrix")
display(pd.crosstab(y_test_cat, y_pred, rownames=['True'], colnames=['Predicted'], margins=True))

****** Test Data ********
             precision    recall  f1-score   support

          0       0.93      0.99      0.96       289
          1       0.99      0.98      0.98       355
          2       0.93      0.94      0.94       303
          3       0.97      0.94      0.95       303
          4       0.95      0.96      0.95       269
          5       0.94      0.96      0.95       286
          6       0.93      0.97      0.95       310
          7       0.95      0.96      0.96       299
          8       0.97      0.92      0.94       287
          9       0.97      0.92      0.95       299

avg / total       0.95      0.95      0.95      3000

Confusion Matrix


Predicted,0,1,2,3,4,5,6,7,8,9,All
True,,,,,,,,,,,
0,285,0,0,0,0,0,2,0,2,0,289
1,0,348,2,1,0,1,0,1,1,1,355
2,4,2,284,3,3,0,3,2,2,0,303
3,0,0,5,284,0,10,0,3,0,1,303
4,1,1,0,0,259,0,5,0,0,3,269
5,1,0,1,1,2,275,4,0,1,1,286
6,3,0,0,0,1,5,301,0,0,0,310
7,1,1,6,1,1,0,0,287,0,2,299
8,6,0,4,0,2,1,8,2,263,1,287


Draw the model

In [11]:
SVG(model_to_dot(model).create(prog='dot', format='svg'))

ImportError: Failed to import `pydot`. Please install `pydot`. For example with `pip install pydot`.